# Day 8: Treetop Tree House

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these 
trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible
from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:

```
30373
25512
65332
33549
35390
```

Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; 
that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, 
that only leaves the interior nine trees to consider:

- The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
- The top-middle 5 is visible from the top and right.
- The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
- The left-middle 5 is visible, but only from the right.
- The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
- The right-middle 3 is visible from the right.
- In the bottom row, the middle 5 is visible, but the 3 and 4 are not.

With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?

In [55]:
#@title Define basic functions

def generate_grid(filename: str):
    grid = []
    with open(filename) as f:
        for line in f:
            grid_row = []
            trimmed_line = line.rstrip('\n')
            for c in trimmed_line:
                grid_row.append(int(c))
            grid.append(grid_row)

    return grid            

def calculate_perimeter(width : int, height: int) -> int:
    return (width * 2) + ((height - 2) * 2)

def ranges_to_search(grid, row_idx: int, col_idx: int):
    nrange = []
    srange = []
    wrange = []
    erange = []
    
    for i in range(len(grid[0])):
        if i < col_idx: 
            wrange.insert(0, [row_idx, i])
        elif i == col_idx:
            continue
        else:
            erange.append([row_idx, i])
    
    for i in range(len(grid[0])):
        if i < row_idx: 
            nrange.insert(0, [i, col_idx])
        elif i == row_idx:
            continue
        else:
            srange.append([i, col_idx])

    return [nrange, srange, wrange, erange]

def print_range(grid, range):
    result = ''
    for r in range:
        result = result + f'[{r[0]}][{r[1]}]="{grid[r[0]][r[1]]}" '
    return result

def eval_range(grid, range):
    result = []
    for r in range:
        result.append(grid[r[0]][r[1]])
    return result

def check_if_visible(grid, row_idx: int, col_idx: int) -> bool:
    whatami = grid[row_idx][col_idx]
    ranges = ranges_to_search(grid, row_idx, col_idx)

    for r in ranges:
        visible = True
        for v in eval_range(grid, r):
            if v >= whatami:
                visible = False
        if visible:
            return True

    return False

# basic brute force approach
def calculate_visible(grid):
    rows = len(grid)
    cols = len(grid[0])
    total = calculate_perimeter(cols, rows)
    for row_idx in range(1, rows - 1):
        for col_idx in range(1, cols - 1):
            if check_if_visible(grid, row_idx, col_idx):
                total = total + 1

    return total

In [56]:
#@title write a test

import unittest

class TestProcessGrid(unittest.TestCase):

    def test_generate_grid(self):
        expected = [
            [3, 0, 3, 7, 3],
            [2, 5, 5, 1, 2],
            [6, 5, 3, 3, 2],
            [3, 3, 5, 4, 9],
            [3, 5, 3, 9, 0],
        ]

        actual = generate_grid('test.txt')
        
        self.assertEqual(actual, expected)

    def test_calculate_perimeter(self):
        self.assertEqual(calculate_perimeter(5, 5), 16)

    def test_ranges_to_search(self):
        grid = [
            [3, 0, 3, 7, 3],
            [2, 5, 5, 1, 2],
            [6, 5, 3, 3, 2],
            [3, 3, 5, 4, 9],
            [3, 5, 3, 9, 0],
        ]
        actual = ranges_to_search(grid, 1, 1)
        expected = [
            [[0,1]],
            [[2,1], [3,1], [4,1]],
            [[1,0]],
            [[1,2], [1,3], [1,4]],
        ]
        self.assertEqual(actual, expected)


    def test_check_if_visible(self):
        grid = [
            [3, 0, 3, 7, 3],
            [2, 5, 5, 1, 2],
            [6, 5, 3, 3, 2],
            [3, 3, 5, 4, 9],
            [3, 5, 3, 9, 0],
        ]
        
        self.assertEqual(check_if_visible(grid, 1, 1), True)
        self.assertEqual(check_if_visible(grid, 1, 3), False)
        
    def test_calculate_visible(self):
        input = [
            [3, 0, 3, 7, 3],
            [2, 5, 5, 1, 2],
            [6, 5, 3, 3, 2],
            [3, 3, 5, 4, 9],
            [3, 5, 3, 9, 0],
        ]
        
        self.assertEqual(calculate_visible(input), 21)
                         
unittest.main(argv=[''], verbosity=2, exit=False)

test_calculate_perimeter (__main__.TestProcessGrid.test_calculate_perimeter) ... ok
test_calculate_visible (__main__.TestProcessGrid.test_calculate_visible) ... ok
test_check_if_visible (__main__.TestProcessGrid.test_check_if_visible) ... ok
test_generate_grid (__main__.TestProcessGrid.test_generate_grid) ... ok
test_ranges_to_search (__main__.TestProcessGrid.test_ranges_to_search) ... ok
test_scenic_score (__main__.TestScenicScore.test_scenic_score) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.011s

OK


Score of 1 for 5 with range values: [3]
Score of 2 for 5 with range values: [3, 5, 3]
Score of 1 for 5 with range values: [5, 2]
Score of 2 for 5 with range values: [1, 2]


In [57]:
puzzle_grid = generate_grid('input.txt')
trees = calculate_visible(puzzle_grid)
print(f'Tree total = {trees}')

Tree total = 1533


Your puzzle answer was 1533.

The first half of this puzzle is complete! It provides one gold star: *

# Part Two

Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of trees.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large eaves to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle 5 in the second row:

```
30373
25512
65332
33549
35390
```

- Looking up, its view is not blocked; it can see 1 tree (of height 3).
- Looking left, its view is blocked immediately; it can see only 1 tree (of height 5, right next to it).
- Looking right, its view is not blocked; it can see 2 trees.
- Looking down, its view is blocked eventually; it can see 2 trees (one of height 3, then the tree of height 5 that blocks its view).

A tree's scenic score is found by multiplying together its viewing distance in each of the four directions. For this tree, this is 4 (found by multiplying 1 * 1 * 2 * 2).

However, you can do even better: consider the tree of height 5 in the middle of the fourth row:

```
30373
25512
65332
33549
35390
```

- Looking up, its view is blocked at 2 trees (by another tree with a height of 5).
- Looking left, its view is not blocked; it can see 2 trees.
- Looking down, its view is also not blocked; it can see 1 tree.
- Looking right, its view is blocked at 2 trees (by a massive tree of height 9).

This tree's scenic score is 8 (2 * 2 * 1 * 2); this is the ideal spot for the tree house.

Consider each tree on your map. What is the highest scenic score possible for any tree?

In [62]:
from typing import List

def individual_scenic(range_values: List[int], whatami : int):
    view = 0
    for r in range_values:
        view = view + 1
        if r >= whatami:
            break
            
    #print(f'Score of {view} for {whatami} with range values: {range_values}')
    
    return view
        

def scenic_score(grid, row_idx: int, col_idx: int) -> int:
    score = 1
    whatami = grid[row_idx][col_idx]
    ranges = ranges_to_search(grid, row_idx, col_idx)
    
    for r in ranges:
        score = score * individual_scenic(eval_range(grid, r), whatami)

    return score



In [63]:
#@title Scenic score test

import unittest

class TestScenicScore(unittest.TestCase):

    def test_scenic_score(self):
        grid = [
            [3, 0, 3, 7, 3],
            [2, 5, 5, 1, 2],
            [6, 5, 3, 3, 2],
            [3, 3, 5, 4, 9],
            [3, 5, 3, 9, 0],
        ]
        
        self.assertEqual(scenic_score(grid, 1, 2), 4)
        self.assertEqual(scenic_score(grid, 3, 2), 8)
                         
unittest.main(argv=[''], verbosity=2, exit=False)

test_calculate_perimeter (__main__.TestProcessGrid.test_calculate_perimeter) ... ok
test_calculate_visible (__main__.TestProcessGrid.test_calculate_visible) ... ok
test_check_if_visible (__main__.TestProcessGrid.test_check_if_visible) ... ok
test_generate_grid (__main__.TestProcessGrid.test_generate_grid) ... ok
test_ranges_to_search (__main__.TestProcessGrid.test_ranges_to_search) ... ok
test_scenic_score (__main__.TestScenicScore.test_scenic_score) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.013s

OK


In [64]:
max_score = 0
rows = len(puzzle_grid)
cols = len(puzzle_grid[0])

for row_idx in range(1, rows):
    for col_idx in range(1, cols - 1):
        scenic = scenic_score(puzzle_grid, row_idx, col_idx)
        max_score = max(scenic, max_score)

print(f'Max scenic score = {max_score}')
                              

Max scenic score = 345744
